#### Annual Sales Tax Revenue Analysis

### [Data Source](https://tax.utah.gov/econstats/sales)

### Cal Year

### Notes:


In [1]:
import pandas as pd
import os
import numpy as np
import arcpy
from arcgis.features import GeoAccessor, GeoSeriesAccessor
arcpy.env.overwriteOutput = True

In [2]:
# show all columns
pd.options.display.max_columns = None

In [3]:
# Removes misc characters from string
def replace(string):
    string = string.replace("&", "")
    string = string.replace("-", " ")
    string = string.replace("(", "")
    string = string.replace(")", "")
    string = string.replace("/", " ")
    string = string.replace(",", "")
    string = string.replace(".", "")
    string = string.replace(" ", "_")
    return string


### Load Cal Year 2020
2020 sales tax revenue data

In [4]:
# load excel sheet into a dataframe
xlsx = '.\\Inputs\\2020-annual-sales.xlsx'
calyr_2020 = pd.read_excel(xlsx, sheet_name='Table 9', header=5)

# Drop last empty two rows
calyr_2020.drop(calyr_2020.tail(2).index,inplace=True)

# forward fill values from merged cells
calyr_2020['County'] = calyr_2020['County'].fillna(method='ffill')
calyr_2020['Location Code'] = calyr_2020['Location Code'].fillna(method='ffill')
calyr_2020['City'] = calyr_2020['City'].fillna(method='ffill')

# Figures with less than 10 taxpayers have been rounded up per Tax Commission disclosure rules.
# Rename column and convert values to boolean
calyr_2020.rename(columns={'Unnamed: 5':'Rounded Up'}, inplace=True)
crosswalk = {'*':True, np.nan:False} # County_ID
calyr_2020['Rounded Up'] = calyr_2020['Rounded Up'].map(crosswalk)

calyr_2020.head()


,County,Location Code,City,Economic Sector (NAICS Code),CY2020,Rounded Up
0,Beaver County,1002.0,Beaver City,"AGRICULTURE, FORESTRY, FISHING & HUNTING (11)",500000.0,True
1,Beaver County,1002.0,Beaver City,"MINING, QUARRYING, & OIL & GAS EXTRACTION (21)",3000.0,True
2,Beaver County,1002.0,Beaver City,UTILITIES (22),2750000.0,True
3,Beaver County,1002.0,Beaver City,CONSTRUCTION (23),258778.0,False
4,Beaver County,1002.0,Beaver City,MANUFACTURING (31-33),6259592.0,False


In [5]:
# pivot the table to get sectors as columns with tax values by city
sectors_calyr_2020 = pd.pivot_table(calyr_2020,values = 'CY2020',index ='City', columns = 'Economic Sector (NAICS Code)', 
                                 aggfunc='first')

# fill NAs with 0
sectors_calyr_2020 = sectors_calyr_2020.fillna(0)

In [6]:
sectors_calyr_2020.columns

Index(['ACCOMMODATION (721)',
       'ADMIN. & SUPPORT & WASTE MANAG. & REMED. SERVICES (56)',
       'AGRICULTURE, FORESTRY, FISHING & HUNTING (11)',
       'ARTS, ENTERTAINMENT AND RECREATION (71)', 'CONSTRUCTION (23)',
       'EDUCATIONAL SERVICES (61)', 'FINANCE & INSURANCE (52)',
       'FOOD SERVICES & DRINKING PLACES (722)',
       'HEALTH CARE & SOCIAL ASSISTANCE (62)', 'INFORMATION (51)',
       'MANAGEMENT OF COMPANIES & ENTERPRISES (55)', 'MANUFACTURING (31-33)',
       'MINING, QUARRYING, & OIL & GAS EXTRACTION (21)',
       'OTHER SERVICES-EXCEPT PUBLIC ADMINISTRATION (81)',
       'PRIOR-PERIOD PAYMENTS & REFUNDS', 'PRIVATE MOTOR VEHICLE SALES',
       'PROFESSIONAL, SCIENTIFIC & TECHNICAL SERVICES (54)',
       'PUBLIC ADMINISTRATION (92)', 'REAL ESTATE, RENTAL & LEASING (53)',
       'RETAIL-BUILD. MATERIAL, GARDEN EQUIP. & SUPPLIES DEALERS (444)',
       'RETAIL-CLOTHING & CLOTHING ACCESSORIES STORES (448)',
       'RETAIL-ELECTRONICS & APPLIANCE STORES (443)',
       

In [7]:
# fix column names
#new_names = [replace(item) for item in list(sectors_q4_2020.columns)]
new_names = ['L_ACCOMMODATION_20', 'S_ADMIN_SUPPORT_20','A_AG_WILDLIFE_20','L_CULTURAL_REC_20','C_CONSTRUCTION_20',
'E_EDUCATION_20','S_FINANCIAL_20','L_RSTRNT_BAR_20','H_HEALTH_CARE_20','S_IT_DATA_20','S_CORPORATE_MGMT_20',
'M_MANUFACTURING_20','E_EXTRACTION_20','O_OTHER_20','X_ADJUSTMENTS_20','R_AUTO_PRIVATE_20','S_PROF_TECH_SRV_20',
'G_GOVERNMENT_20','S_RENTAL_LEASING_20','R_BUILDING_SUPPLY_20','R_CLOTHING_20','R_ELECTRONICS_20','R_GROCERY_BEV_20',
'R_FURNITURE_20','R_GAS_STATIONS_20','R_GENERAL_RETAIL_20','R_HEALTH_RETAIL_20','R_OTHER_RETAIL_20','R_AUTO_RETAIL_20',
'R_NONSTORE_RETAIL_20','R_SPORT_HOBBY_20','L_SPECIAL_EVENT_20','W_DISTRIBUTION_20','X_UNKNOWN_20','U_UTILITIES_20',
'W_WHLSALE_DURABLE_20','S_WHLSLE_ETRADE_20','W_WHLSLE_NDURABLE_20']
sectors_calyr_2020.columns = new_names

# get the total sales tax revenue
sectors_calyr_2020['TOTAL_20'] = sectors_calyr_2020[new_names].sum(axis=1)

sectors_calyr_2020.head()

,L_ACCOMMODATION_20,S_ADMIN_SUPPORT_20,A_AG_WILDLIFE_20,L_CULTURAL_REC_20,C_CONSTRUCTION_20,E_EDUCATION_20,S_FINANCIAL_20,L_RSTRNT_BAR_20,H_HEALTH_CARE_20,S_IT_DATA_20,S_CORPORATE_MGMT_20,M_MANUFACTURING_20,E_EXTRACTION_20,O_OTHER_20,X_ADJUSTMENTS_20,R_AUTO_PRIVATE_20,S_PROF_TECH_SRV_20,G_GOVERNMENT_20,S_RENTAL_LEASING_20,R_BUILDING_SUPPLY_20,R_CLOTHING_20,R_ELECTRONICS_20,R_GROCERY_BEV_20,R_FURNITURE_20,R_GAS_STATIONS_20,R_GENERAL_RETAIL_20,R_HEALTH_RETAIL_20,R_OTHER_RETAIL_20,R_AUTO_RETAIL_20,R_NONSTORE_RETAIL_20,R_SPORT_HOBBY_20,L_SPECIAL_EVENT_20,W_DISTRIBUTION_20,X_UNKNOWN_20,U_UTILITIES_20,W_WHLSALE_DURABLE_20,S_WHLSLE_ETRADE_20,W_WHLSLE_NDURABLE_20,TOTAL_20
City,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
American Fork,1878400.0,2839307.0,2875138.0,5931456.0,39603098.0,442370.0,10592193.0,104349884.0,1898850.0,31899515.0,60000.0,14442382.0,350000.0,47243005.0,9767823.0,18000000.0,20816225.0,200000.0,15322550.0,94422620.0,28914533.0,26609471.0,17067144.0,16732105.0,11000000.0,185494692.0,26696378.0,31290958.0,405153128.0,77393079.0,36776931.0,200000.0,2208045.0,0.0,27500000.0,62330946.0,1964156.0,30445939.0,1.410712e+09
Beaver City,4912871.0,82679.0,500000.0,30000.0,258778.0,46482.0,1714929.0,10127155.0,250000.0,2876142.0,8000.0,6259592.0,3000.0,3905297.0,840675.0,2000000.0,586751.0,0.0,582360.0,3335406.0,473077.0,128134.0,7568481.0,49329.0,14750000.0,2098171.0,545807.0,942391.0,3261778.0,6613252.0,186949.0,45000.0,30000.0,0.0,2750000.0,1447110.0,214563.0,586935.0,8.001109e+07
Blanding,2075502.0,168578.0,1000.0,150000.0,878937.0,322791.0,76907.0,1710742.0,200000.0,1815107.0,8000.0,855953.0,1000.0,1787098.0,-345158.0,3000000.0,545856.0,2000000.0,246818.0,8863939.0,362055.0,442393.0,8000000.0,57899.0,6000000.0,2828926.0,316884.0,503641.0,1466659.0,6163988.0,311028.0,0.0,70000.0,0.0,400000.0,1374938.0,30911.0,1608143.0,5.430054e+07
Bountiful,700000.0,6271951.0,25681.0,3904858.0,7206423.0,455012.0,4710201.0,50325246.0,4957256.0,23983623.0,90000.0,10655385.0,90000.0,25398674.0,8551645.0,18250000.0,15004807.0,2000000.0,8367435.0,7433117.0,12605951.0,16355255.0,99440715.0,7156565.0,9750000.0,9590590.0,9178325.0,15304955.0,207126274.0,74190593.0,8698662.0,200000.0,119744.0,1000.0,35000000.0,22580969.0,481746.0,3114586.0,7.292772e+08
Brigham,3657247.0,765247.0,8598.0,1838897.0,3715796.0,204223.0,1131414.0,27683037.0,1427040.0,11221430.0,30000.0,30249675.0,15000.0,14024448.0,3115698.0,10250000.0,3180936.0,6000.0,7562648.0,9629247.0,2060819.0,5633194.0,48161414.0,2217817.0,13495929.0,2822539.0,1603841.0,4117706.0,64456378.0,25883963.0,2165774.0,30000.0,304508.0,0.0,13250000.0,26205132.0,443321.0,1221925.0,3.437908e+08


#### Load Q4 2019 sales tax revenue data

In [8]:
# load excel sheet into a dataframe
xlsx = '.\\Inputs\\2019-annual-sales.xlsx'
calyr_2019 = pd.read_excel(xlsx, sheet_name='Table 9', header=5)

# Drop last empty two rows
calyr_2019.drop(calyr_2019.tail(2).index,inplace=True)

# forward fill values from merged cells
calyr_2019['County'] = calyr_2019['County'].fillna(method='ffill')
calyr_2019['Location Code'] = calyr_2019['Location Code'].fillna(method='ffill')
calyr_2019['City'] = calyr_2019['City'].fillna(method='ffill')

# Figures with less than 10 taxpayers have been rounded up per Tax Commission disclosure rules.
# Rename column and convert values to boolean
calyr_2019.rename(columns={'Unnamed: 5':'Rounded Up'}, inplace=True)
crosswalk = {'*':True, np.nan:False} # County_ID
calyr_2019['Rounded Up'] = calyr_2019['Rounded Up'].map(crosswalk)

calyr_2019.tail()

,County,Location Code,City,Economic Sector (NAICS Code),CY2019,Rounded Up
2445,Weber County,29040.0,South Ogden,OTHER SERVICES-EXCEPT PUBLIC ADMINISTRATION (81),5865210.0,False
2446,Weber County,29040.0,South Ogden,PUBLIC ADMINISTRATION (92),7000.0,True
2447,Weber County,29040.0,South Ogden,PRIVATE MOTOR VEHICLE SALES,5000000.0,True
2448,Weber County,29040.0,South Ogden,SPECIAL EVENT SALES,80000.0,True
2449,Weber County,29040.0,South Ogden,PRIOR-PERIOD PAYMENTS & REFUNDS,1648584.0,False


In [9]:
# pivot the table to get sectors as columns with tax values by city
sectors_calyr_2019 = pd.pivot_table(calyr_2019,values = 'CY2019',index ='City', columns = 'Economic Sector (NAICS Code)', 
                                 aggfunc='first')

# fill NAs with 0
sectors_calyr_2019 = sectors_calyr_2019.fillna(0)

# fix column names
#new_names = [replace(item) for item in list(sectors_q4_2019.columns)]
new_names = ['L_ACCOMMODATION_19', 'S_ADMIN_SUPPORT_19','A_AG_WILDLIFE_19','L_CULTURAL_REC_19','C_CONSTRUCTION_19',
'E_EDUCATION_19','S_FINANCIAL_19','L_RSTRNT_BAR_19','H_HEALTH_CARE_19','S_IT_DATA_19','S_CORPORATE_MGMT_19',
'M_MANUFACTURING_19','E_EXTRACTION_19','O_OTHER_19','X_ADJUSTMENTS_19','R_AUTO_PRIVATE_19','S_PROF_TECH_SRV_19',
'G_GOVERNMENT_19','S_RENTAL_LEASING_19','R_BUILDING_SUPPLY_19','R_CLOTHING_19','R_ELECTRONICS_19','R_GROCERY_BEV_19',
'R_FURNITURE_19','R_GAS_STATIONS_19','R_GENERAL_RETAIL_19','R_HEALTH_RETAIL_19','R_OTHER_RETAIL_19','R_AUTO_RETAIL_19',
'R_NONSTORE_RETAIL_19','R_SPORT_HOBBY_19','L_SPECIAL_EVENT_19','W_DISTRIBUTION_19','X_UNKNOWN_19','U_UTILITIES_19',
'W_WHLSALE_DURABLE_19','S_WHLSLE_ETRADE_19','W_WHLSLE_NDURABLE_19']
sectors_calyr_2019.columns = new_names

# drop unknown column since its not present in 2020
#sectors_q4_2019.drop('X_UNKNOWN_19', axis=1, inplace=True)
#new_names.remove('X_UNKNOWN_19')

# get the total sales tax revenue
sectors_calyr_2019['TOTAL_19'] = sectors_calyr_2019[new_names].sum(axis=1)

sectors_calyr_2019.head()

,L_ACCOMMODATION_19,S_ADMIN_SUPPORT_19,A_AG_WILDLIFE_19,L_CULTURAL_REC_19,C_CONSTRUCTION_19,E_EDUCATION_19,S_FINANCIAL_19,L_RSTRNT_BAR_19,H_HEALTH_CARE_19,S_IT_DATA_19,S_CORPORATE_MGMT_19,M_MANUFACTURING_19,E_EXTRACTION_19,O_OTHER_19,X_ADJUSTMENTS_19,R_AUTO_PRIVATE_19,S_PROF_TECH_SRV_19,G_GOVERNMENT_19,S_RENTAL_LEASING_19,R_BUILDING_SUPPLY_19,R_CLOTHING_19,R_ELECTRONICS_19,R_GROCERY_BEV_19,R_FURNITURE_19,R_GAS_STATIONS_19,R_GENERAL_RETAIL_19,R_HEALTH_RETAIL_19,R_OTHER_RETAIL_19,R_AUTO_RETAIL_19,R_NONSTORE_RETAIL_19,R_SPORT_HOBBY_19,L_SPECIAL_EVENT_19,W_DISTRIBUTION_19,X_UNKNOWN_19,U_UTILITIES_19,W_WHLSALE_DURABLE_19,S_WHLSLE_ETRADE_19,W_WHLSLE_NDURABLE_19,TOTAL_19
City,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
American Fork,2537613.0,2552910.0,1750000.0,6467601.0,32735467.0,416107.0,10128930.0,97383158.0,1918417.0,38370756.0,60000.0,10741593.0,250000.0,43377648.0,11906936.0,11250000.0,17729496.0,90000.0,15136893.0,77332237.0,30782170.0,32879118.0,16177423.0,15578247.0,10994687.0,168866049.0,23341790.0,25410309.0,363165149.0,42351624.0,33333050.0,604662.0,2032544.0,0.0,30250000.0,46167765.0,87299.0,24508447.0,1.248666e+09
Beaver City,4805537.0,103632.0,300000.0,30000.0,360477.0,68551.0,1190849.0,9820732.0,200000.0,2380564.0,6000.0,4461526.0,0.0,3456681.0,628826.0,1750000.0,400435.0,0.0,399192.0,2207222.0,309900.0,104804.0,6500000.0,25928.0,13250000.0,2339236.0,327358.0,1138954.0,2876651.0,3354613.0,137197.0,25000.0,15000.0,7000.0,2750000.0,1488429.0,80000.0,772165.0,6.807246e+07
Blanding,4043564.0,204922.0,2000.0,250000.0,815000.0,467154.0,86263.0,2250000.0,250000.0,2174354.0,15000.0,609232.0,0.0,1079534.0,1697521.0,3000000.0,366851.0,2000000.0,363550.0,6598780.0,264690.0,312800.0,7000000.0,35319.0,4750000.0,3024491.0,260366.0,551086.0,1243972.0,3449583.0,243120.0,5000.0,80000.0,0.0,700000.0,1715943.0,7000.0,1312716.0,5.122981e+07
Bountiful,700000.0,9536828.0,6000.0,4409965.0,7437407.0,455409.0,4872026.0,52253762.0,4107293.0,20506638.0,100000.0,10940271.0,70000.0,26445564.0,11073265.0,17500000.0,10914337.0,1500000.0,9182908.0,4746627.0,12359267.0,14292963.0,89495871.0,6756247.0,9250000.0,7860661.0,7570279.0,13089947.0,212382704.0,40888020.0,6280114.0,150000.0,158047.0,0.0,32750000.0,17808411.0,48134.0,2689587.0,6.705886e+08
Brigham,5887485.0,763477.0,5000.0,1993021.0,5949405.0,239569.0,1145653.0,26906567.0,1666410.0,11071258.0,70000.0,26059670.0,10000.0,12630258.0,4451055.0,8250000.0,2059740.0,8000.0,5932594.0,7006638.0,1857215.0,4844330.0,41907498.0,2058474.0,12522195.0,4675853.0,1595325.0,5158982.0,58400426.0,15469728.0,1740702.0,504956.0,284636.0,0.0,12750000.0,16788624.0,38263.0,1360831.0,3.040638e+08


### Calculate the differences 
*2020 - 2019 = Diff*

In [10]:
# Check join output shape
print(sectors_calyr_2019.shape)
print(sectors_calyr_2020.shape)

(66, 39)
(66, 39)


In [11]:
# merge the 2019-2020 difference with the sdf
sectors_calyr_diff = pd.DataFrame(sectors_calyr_2020.values - sectors_calyr_2019.values)

# add back the column names
new_names = ['L_ACCOMMODATION_D20', 'S_ADMIN_SUPPORT_D20','A_AG_WILDLIFE_D20','L_CULTURAL_REC_D20','C_CONSTRUCTION_D20',
'E_EDUCATION_D20','S_FINANCIAL_D20','L_RSTRNT_BAR_D20','H_HEALTH_CARE_D20','S_IT_DATA_D20','S_CORPORATE_MGMT_D20',
'M_MANUFACTURING_D20','E_EXTRACTION_D20','O_OTHER_D20','X_ADJUSTMENTS_D20','R_AUTO_PRIVATE_D20','S_PROF_TECH_SRV_D20',
'G_GOVERNMENT_D20','S_RENTAL_LEASING_D20','R_BUILDING_SUPPLY_D20','R_CLOTHING_D20','R_ELECTRONICS_D20','R_GROCERY_BEV_D20',
'R_FURNITURE_D20','R_GAS_STATIONS_D20','R_GENERAL_RETAIL_D20','R_HEALTH_RETAIL_D20','R_OTHER_RETAIL_D20','R_AUTO_RETAIL_D20',
'R_NONSTORE_RETAIL_D20','R_SPORT_HOBBY_D20','L_SPECIAL_EVENT_D20','W_DISTRIBUTION_D20','X_UNKNOWN_D20','U_UTILITIES_D20',
'W_WHLSALE_DURABLE_D20','S_WHLSLE_ETRADE_D20','W_WHLSLE_NDURABLE_D20', 'TOTAL_D20']

sectors_calyr_diff.columns = new_names

# add back cities as index
sectors_calyr_diff['City'] = sectors_calyr_2020.index
sectors_calyr_diff = sectors_calyr_diff.set_index('City')

#check table
sectors_calyr_diff.head(10)

,L_ACCOMMODATION_D20,S_ADMIN_SUPPORT_D20,A_AG_WILDLIFE_D20,L_CULTURAL_REC_D20,C_CONSTRUCTION_D20,E_EDUCATION_D20,S_FINANCIAL_D20,L_RSTRNT_BAR_D20,H_HEALTH_CARE_D20,S_IT_DATA_D20,S_CORPORATE_MGMT_D20,M_MANUFACTURING_D20,E_EXTRACTION_D20,O_OTHER_D20,X_ADJUSTMENTS_D20,R_AUTO_PRIVATE_D20,S_PROF_TECH_SRV_D20,G_GOVERNMENT_D20,S_RENTAL_LEASING_D20,R_BUILDING_SUPPLY_D20,R_CLOTHING_D20,R_ELECTRONICS_D20,R_GROCERY_BEV_D20,R_FURNITURE_D20,R_GAS_STATIONS_D20,R_GENERAL_RETAIL_D20,R_HEALTH_RETAIL_D20,R_OTHER_RETAIL_D20,R_AUTO_RETAIL_D20,R_NONSTORE_RETAIL_D20,R_SPORT_HOBBY_D20,L_SPECIAL_EVENT_D20,W_DISTRIBUTION_D20,X_UNKNOWN_D20,U_UTILITIES_D20,W_WHLSALE_DURABLE_D20,S_WHLSLE_ETRADE_D20,W_WHLSLE_NDURABLE_D20,TOTAL_D20
City,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
American Fork,-659213.0,286397.0,1125138.0,-536145.0,6867631.0,26263.0,463263.0,6966726.0,-19567.0,-6471241.0,0.0,3700789.0,100000.0,3865357.0,-2139113.0,6750000.0,3086729.0,110000.0,185657.0,17090383.0,-1867637.0,-6269647.0,889721.0,1153858.0,5313.0,16628643.0,3354588.0,5880649.0,41987979.0,35041455.0,3443881.0,-404662.0,175501.0,0.0,-2750000.0,16163181.0,1876857.0,5937492.0,162046226.0
Beaver City,107334.0,-20953.0,200000.0,0.0,-101699.0,-22069.0,524080.0,306423.0,50000.0,495578.0,2000.0,1798066.0,3000.0,448616.0,211849.0,250000.0,186316.0,0.0,183168.0,1128184.0,163177.0,23330.0,1068481.0,23401.0,1500000.0,-241065.0,218449.0,-196563.0,385127.0,3258639.0,49752.0,20000.0,15000.0,-7000.0,0.0,-41319.0,134563.0,-185230.0,11938635.0
Blanding,-1968062.0,-36344.0,-1000.0,-100000.0,63937.0,-144363.0,-9356.0,-539258.0,-50000.0,-359247.0,-7000.0,246721.0,1000.0,707564.0,-2042679.0,0.0,179005.0,0.0,-116732.0,2265159.0,97365.0,129593.0,1000000.0,22580.0,1250000.0,-195565.0,56518.0,-47445.0,222687.0,2714405.0,67908.0,-5000.0,-10000.0,0.0,-300000.0,-341005.0,23911.0,295427.0,3070724.0
Bountiful,0.0,-3264877.0,19681.0,-505107.0,-230984.0,-397.0,-161825.0,-1928516.0,849963.0,3476985.0,-10000.0,-284886.0,20000.0,-1046890.0,-2521620.0,750000.0,4090470.0,500000.0,-815473.0,2686490.0,246684.0,2062292.0,9944844.0,400318.0,500000.0,1729929.0,1608046.0,2215008.0,-5256430.0,33302573.0,2418548.0,50000.0,-38303.0,1000.0,2250000.0,4772558.0,433612.0,424999.0,58688692.0
Brigham,-2230238.0,1770.0,3598.0,-154124.0,-2233609.0,-35346.0,-14239.0,776470.0,-239370.0,150172.0,-40000.0,4190005.0,5000.0,1394190.0,-1335357.0,2000000.0,1121196.0,-2000.0,1630054.0,2622609.0,203604.0,788864.0,6253916.0,159343.0,973734.0,-1853314.0,8516.0,-1041276.0,6055952.0,10414235.0,425072.0,-474956.0,19872.0,0.0,500000.0,9416508.0,405058.0,-138906.0,39727003.0
Cedar City,-4223407.0,2000550.0,7625.0,-92623.0,1798180.0,-905809.0,-764229.0,3980672.0,114207.0,878024.0,-10000.0,8404331.0,160000.0,3561588.0,-28047221.0,5250000.0,2779250.0,0.0,1432010.0,33585031.0,270890.0,3018402.0,13393465.0,4798874.0,5019144.0,17937247.0,733419.0,5654433.0,9355265.0,25477275.0,7082924.0,551811.0,-3373326.0,0.0,1750000.0,11901568.0,666139.0,-263911.0,133881798.0
Centerville,20000.0,236928.0,-2253.0,53889.0,-928230.0,48168.0,-636127.0,828467.0,-40606.0,-5914632.0,-380000.0,204004.0,-10000.0,1530455.0,655657.0,1750000.0,1722651.0,-20000.0,484358.0,11212194.0,91941.0,-366045.0,2920261.0,1334856.0,750000.0,3667501.0,102938.0,-2661217.0,-1364684.0,12685795.0,1035778.0,692000.0,8398.0,0.0,250000.0,2221400.0,314903.0,-26159.0,32472589.0
Clearfield,-235020.0,-30012.0,6393.0,327405.0,1317343.0,24350.0,-22082.0,584800.0,-1948.0,2824892.0,750000.0,-1905626.0,120000.0,338340.0,-889011.0,1500000.0,4140330.0,-200000.0,1033750.0,1655575.0,1281702.0,16340.0,1573062.0,103820.0,3613643.0,1131735.0,247039.0,2223945.0,1262779.0,23100626.0,660876.0,175000.0,-21788.0,0.0,-5250000.0,4212244.0,589332.0,-258798.0,46001036.0
Cottonwood Heights,-183999.0,259164.0,7711.0,-3376888.0,535984.0,-43283.0,-472220.0,-10073738.0,-278704.0,3946154.0,-250000.0,-1852073.0,-115000.0,171600.0,-1881885.0,1750000.0,7004677.0,-500000.0,-533281.0,7768931.0,-1174329.0,-2692432.0,10141211.0,450022.0,757682

### Load municipalities and townships shapefile and join to formatted sales tax data

In [13]:
# load cities shapefile into pandas spatial dataframe
cities_shp = '.\\Inputs\\Cities.shp'
cities_sdf = pd.DataFrame.spatial.from_featureclass(cities_shp)

# Rename columns
cities_sdf.columns = ['FID', 'NAME', 'SOURCE', 'SALESTAXID', 'POPLASTCEN', 'POPLASTEST',
       'AREA_SQMI', 'SHAPE']

In [14]:
# merge the 2020 sdf with the pivoted data
cities_sdf2 = cities_sdf.merge(sectors_calyr_2019, left_on= "NAME", right_on="City", how="inner")
cities_sdf2 = cities_sdf2.merge(sectors_calyr_2020, left_on= "NAME", right_on="City", how="inner")
cities_sdf2 = cities_sdf2.merge(sectors_calyr_diff, left_on= "NAME", right_on="City", how="inner")
cities_sdf2.tail(10)


,FID,NAME,SOURCE,SALESTAXID,POPLASTCEN,POPLASTEST,AREA_SQMI,SHAPE,L_ACCOMMODATION_19,S_ADMIN_SUPPORT_19,A_AG_WILDLIFE_19,L_CULTURAL_REC_19,C_CONSTRUCTION_19,E_EDUCATION_19,S_FINANCIAL_19,L_RSTRNT_BAR_19,H_HEALTH_CARE_19,S_IT_DATA_19,S_CORPORATE_MGMT_19,M_MANUFACTURING_19,E_EXTRACTION_19,O_OTHER_19,X_ADJUSTMENTS_19,R_AUTO_PRIVATE_19,S_PROF_TECH_SRV_19,G_GOVERNMENT_19,S_RENTAL_LEASING_19,R_BUILDING_SUPPLY_19,R_CLOTHING_19,R_ELECTRONICS_19,R_GROCERY_BEV_19,R_FURNITURE_19,R_GAS_STATIONS_19,R_GENERAL_RETAIL_19,R_HEALTH_RETAIL_19,R_OTHER_RETAIL_19,R_AUTO_RETAIL_19,R_NONSTORE_RETAIL_19,R_SPORT_HOBBY_19,L_SPECIAL_EVENT_19,W_DISTRIBUTION_19,X_UNKNOWN_19,U_UTILITIES_19,W_WHLSALE_DURABLE_19,S_WHLSLE_ETRADE_19,W_WHLSLE_NDURABLE_19,TOTAL_19,L_ACCOMMODATION_20,S_ADMIN_SUPPORT_20,A_AG_WILDLIFE_20,L_CULTURAL_REC_20,C_CONSTRUCTION_20,E_EDUCATION_20,S_FINANCIAL_20,L_RSTRNT_BAR_20,H_HEALTH_CARE_20,S_IT_DATA_20,S_CORPORATE_MGMT_20,M_MANUFACTURING_20,E_EXTRACTION_20,O_OTHER_20,X_ADJUSTMENTS_20,R_AUTO_PRIVATE_20,S_PROF_TECH_SRV_20,G_GOVERNMENT_20,S_RENTAL_LEASING_20,R_BUILDING_SUPPLY_20,R_CLOTHING_20,R_ELECTRONICS_20,R_GROCERY_BEV_20,R_FURNITURE_20,R_GAS_STATIONS_20,R_GENERAL_RETAIL_20,R_HEALTH_RETAIL_20,R_OTHER_RETAIL_20,R_AUTO_RETAIL_20,R_NONSTORE_RETAIL_20,R_SPORT_HOBBY_20,L_SPECIAL_EVENT_20,W_DISTRIBUTION_20,X_UNKNOWN_20,U_UTILITIES_20,W_WHLSALE_DURABLE_20,S_WHLSLE_ETRADE_20,W_WHLSLE_NDURABLE_20,TOTAL_20,L_ACCOMMODATION_D20,S_ADMIN_SUPPORT_D20,A_AG_WILDLIFE_D20,L_CULTURAL_REC_D20,C_CONSTRUCTION_D20,E_EDUCATION_D20,S_FINANCIAL_D20,L_RSTRNT_BAR_D20,H_HEALTH_CARE_D20,S_IT_DATA_D20,S_CORPORATE_MGMT_D20,M_MANUFACTURING_D20,E_EXTRACTION_D20,O_OTHER_D20,X_ADJUSTMENTS_D20,R_AUTO_PRIVATE_D20,S_PROF_TECH_SRV_D20,G_GOVERNMENT_D20,S_RENTAL_LEASING_D20,R_BUILDING_SUPPLY_D20,R_CLOTHING_D20,R_ELECTRONICS_D20,R_GROCERY_BEV_D20,R_FURNITURE_D20,R_GAS_STATIONS_D20,R_GENERAL_RETAIL_D20,R_HEALTH_RETAIL_D20,R_OTHER_RETAIL_D20,R_AUTO_RETAIL_D20,R_NONSTORE_RETAIL_D20,R_SPORT_HOBBY_D20,L_SPECIAL_EVENT_D20,W_DISTRIBUTION_D20,X_UNKNOWN_D20,U_UTILITIES_D20,W_WHLSALE_DURABLE_D20,S_WHLSLE_ETRADE_D20,W_WHLSLE_NDURABLE_D20,TOTAL_D20
55,222,Syracuse,Cities,049,25113,30400,10.193900,"{'rings': [[[411406.60675760085, 4549994.30436...",150000.0,317480.0,4000.0,3848890.0,814638.0,92845.0,1879781.0,18287033.0,152701.0,11396365.0,70000.0,1685139.0,45000.0,5588335.0,3982338.0,12000000.0,2209479.0,40000.0,3380412.0,1897154.0,1780499.0,3302392.0,35899681.0,41258144.0,1750000.0,72699837.0,1554262.0,1973488.0,1828743.0,25369106.0,613648.0,150000.0,25000.0,50000.0,16750000.0,4325760.0,45000.0,947510.0,2.781647e+08,200000.0,773437.0,9725.0,3630811.0,3639648.0,100969.0,1766152.0,19930127.0,197101.0,11546056.0,30000.0,5012385.0,60000.0,5915384.0,3361014.0,16500000.0,3627881.0,20000.0,4252456.0,4248069.0,3015184.0,4223537.0,46540963.0,24663650.0,2000000.0,82940947.0,1958788.0,2718045.0,2602795.0,49799859.0,1237536.0,250000.0,130473.0,0.0,17750000.0,8476531.0,415192.0,1499465.0,3.350442e+08,50000.0,455957.0,5725.0,-218079.0,2825010.0,8124.0,-113629.0,1643094.0,44400.0,149691.0,-40000.0,3327246.0,15000.0,327049.0,-621324.0,4500000.0,1418402.0,-20000.0,872044.0,2350915.0,1234685.0,921145.0,10641282.0,-16594494.0,250000.0,10241110.0,404526.0,744557.0,774052.0,24430753.0,623888.0,100000.0,105473.0,-50000.0,1000000.0,4150771.0,370192.0,551955.0,56879520.0
56,224,Taylorsville,Cities,142,59989,60192,10.839700,"{'rings': [[[422191.826826254, 4504303.3642982...",1500000.0,1935311.0,40000.0,6954210.0,5321911.0,488058.0,8850063.0,111116485.0,522004.0,36946094.0,90000.0,10025874.0,50000.0,8943057.0,8721462.0,15250000.0,7174812.0,3000.0,9218051.0,1808214.0,24935839.0,7959159.0,59418702.0,1748469.0,15250000.0,106426319.0,4620328.0,18746354.0,26295916.0,35163345.0,26927989.0,200000.0,162170.0,0.0,38250000.0,14311659.0,266749.0,1791545.0,6.174331e+08,1500000.0,2301866.0,42260.0,5639004.0,5565025.0,393723.0,8920228.0,108701212.0,655692.0,28131972.0,50000.0,10730420.0,45000.0,8337804.0,8713702.0,17750000.0,1145

In [15]:
# Confirm join count, Salt Lake County (Unincorporated) should be the only one not joined
l1 = list(calyr_2020['City'].value_counts().index)
l2 = list(cities_sdf['NAME'].value_counts().index)
list(set(l1) - set(l2))

['Salt Lake County (Unincorporated)']

In [16]:
# Create file gdb and export sdf to feature class
outputs = '.\\Outputs'

gdb = os.path.join(outputs, "taxable_sales.gdb")
if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs, "taxable_sales.gdb")

cities_sdf2.spatial.to_featureclass(location=os.path.join(outputs, "taxable_sales.gdb","taxable_sales_utah_calyr"))

'C:\\LocalGIS\\GitProjects\\UT_Sales_Tax_Analysis\\Outputs\\taxable_sales.gdb\\taxable_sales_utah_calyr'

### Analysis

In [17]:
cities_sdf2

,FID,NAME,SOURCE,SALESTAXID,POPLASTCEN,POPLASTEST,AREA_SQMI,SHAPE,L_ACCOMMODATION_19,S_ADMIN_SUPPORT_19,A_AG_WILDLIFE_19,L_CULTURAL_REC_19,C_CONSTRUCTION_19,E_EDUCATION_19,S_FINANCIAL_19,L_RSTRNT_BAR_19,H_HEALTH_CARE_19,S_IT_DATA_19,S_CORPORATE_MGMT_19,M_MANUFACTURING_19,E_EXTRACTION_19,O_OTHER_19,X_ADJUSTMENTS_19,R_AUTO_PRIVATE_19,S_PROF_TECH_SRV_19,G_GOVERNMENT_19,S_RENTAL_LEASING_19,R_BUILDING_SUPPLY_19,R_CLOTHING_19,R_ELECTRONICS_19,R_GROCERY_BEV_19,R_FURNITURE_19,R_GAS_STATIONS_19,R_GENERAL_RETAIL_19,R_HEALTH_RETAIL_19,R_OTHER_RETAIL_19,R_AUTO_RETAIL_19,R_NONSTORE_RETAIL_19,R_SPORT_HOBBY_19,L_SPECIAL_EVENT_19,W_DISTRIBUTION_19,X_UNKNOWN_19,U_UTILITIES_19,W_WHLSALE_DURABLE_19,S_WHLSLE_ETRADE_19,W_WHLSLE_NDURABLE_19,TOTAL_19,L_ACCOMMODATION_20,S_ADMIN_SUPPORT_20,A_AG_WILDLIFE_20,L_CULTURAL_REC_20,C_CONSTRUCTION_20,E_EDUCATION_20,S_FINANCIAL_20,L_RSTRNT_BAR_20,H_HEALTH_CARE_20,S_IT_DATA_20,S_CORPORATE_MGMT_20,M_MANUFACTURING_20,E_EXTRACTION_20,O_OTHER_20,X_ADJUSTMENTS_20,R_AUTO_PRIVATE_20,S_PROF_TECH_SRV_20,G_GOVERNMENT_20,S_RENTAL_LEASING_20,R_BUILDING_SUPPLY_20,R_CLOTHING_20,R_ELECTRONICS_20,R_GROCERY_BEV_20,R_FURNITURE_20,R_GAS_STATIONS_20,R_GENERAL_RETAIL_20,R_HEALTH_RETAIL_20,R_OTHER_RETAIL_20,R_AUTO_RETAIL_20,R_NONSTORE_RETAIL_20,R_SPORT_HOBBY_20,L_SPECIAL_EVENT_20,W_DISTRIBUTION_20,X_UNKNOWN_20,U_UTILITIES_20,W_WHLSALE_DURABLE_20,S_WHLSLE_ETRADE_20,W_WHLSLE_NDURABLE_20,TOTAL_20,L_ACCOMMODATION_D20,S_ADMIN_SUPPORT_D20,A_AG_WILDLIFE_D20,L_CULTURAL_REC_D20,C_CONSTRUCTION_D20,E_EDUCATION_D20,S_FINANCIAL_D20,L_RSTRNT_BAR_D20,H_HEALTH_CARE_D20,S_IT_DATA_D20,S_CORPORATE_MGMT_D20,M_MANUFACTURING_D20,E_EXTRACTION_D20,O_OTHER_D20,X_ADJUSTMENTS_D20,R_AUTO_PRIVATE_D20,S_PROF_TECH_SRV_D20,G_GOVERNMENT_D20,S_RENTAL_LEASING_D20,R_BUILDING_SUPPLY_D20,R_CLOTHING_D20,R_ELECTRONICS_D20,R_GROCERY_BEV_D20,R_FURNITURE_D20,R_GAS_STATIONS_D20,R_GENERAL_RETAIL_D20,R_HEALTH_RETAIL_D20,R_OTHER_RETAIL_D20,R_AUTO_RETAIL_D20,R_NONSTORE_RETAIL_D20,R_SPORT_HOBBY_D20,L_SPECIAL_EVENT_D20,W_DISTRIBUTION_D20,X_UNKNOWN_D20,U_UTILITIES_D20,W_WHLSALE_DURABLE_D20,S_WHLSLE_ETRADE_D20,W_WHLSLE_NDURABLE_D20,TOTAL_D20
0,5,American Fork,Cities,002,27413,32519,11.256600,"{""rings"": [[[434244.49682960496, 4467294.37416...",2537613.0,2552910.0,1750000.0,6467601.0,32735467.0,416107.0,10128930.0,97383158.0,1918417.0,38370756.0,60000.0,10741593.0,250000.0,43377648.0,11906936.0,11250000.0,17729496.0,90000.0,15136893.0,77332237.0,30782170.0,32879118.0,16177423.0,15578247.0,10994687.0,168866049.0,23341790.0,25410309.0,363165149.0,42351624.0,33333050.0,604662.0,2032544.0,0.0,30250000.0,46167765.0,87299.0,24508447.0,1.248666e+09,1878400.0,2839307.0,2875138.0,5931456.0,39603098.0,442370.0,10592193.0,104349884.0,1898850.0,31899515.0,60000.0,14442382.0,350000.0,47243005.0,9767823.0,18000000.0,20816225.0,200000.0,15322550.0,94422620.0,28914533.0,26609471.0,17067144.0,16732105.0,11000000.0,185494692.0,26696378.0,31290958.0,405153128.0,77393079.0,36776931.0,200000.0,2208045.0,0.0,27500000.0,62330946.0,1964156.0,30445939.0,1.410712e+09,-659213.0,286397.0,1125138.0,-536145.0,6867631.0,26263.0,463263.0,6966726.0,-19567.0,-6471241.0,0.0,3700789.0,100000.0,3865357.0,-2139113.0,6750000.0,3086729.0,110000.0,185657.0,17090383.0,-1867637.0,-6269647.0,889721.0,1153858.0,5313.0,16628643.0,3354588.0,5880649.0,41987979.0,35041455.0,3443881.0,-404662.0,175501.0,0.0,-2750000.0,16163181.0,1876857.0,5937492.0,162046226.0
1,12,Beaver City,Cities,002,3137,3104,6.638160,"{""rings"": [[[355522.91657235345, 4239879.43388...",4805537.0,103632.0,300000.0,30000.0,360477.0,68551.0,1190849.0,9820732.0,200000.0,2380564.0,6000.0,4461526.0,0.0,3456681.0,628826.0,1750000.0,400435.0,0.0,399192.0,2207222.0,309900.0,104804.0,6500000.0,25928.0,13250000.0,2339236.0,327358.0,1138954.0,2876651.0,3354613.0,137197.0,25000.0,15000.0,7000.0,2750000.0,1488429.0,80000.0,772165.0,6.807246e+07,4912871.0,82679.0,500000.0,30000.0,258778.0,46482.0,1714929.0,10127155.0,250000.0,2876142.0,8000.0,6259592.0,3000.0,3905297.0,84067

In [18]:
# Get year specific column names
standard_cols = ['FID','NAME','SOURCE','SALESTAXID','POPLASTCEN','POPLASTEST','AREA_SQMI', 'SHAPE']
names_19 = standard_cols + [col for col in cities_sdf2.columns if '_19' in col]
names_20 = standard_cols + [col for col in cities_sdf2.columns if '_20' in col]
names_D20 = standard_cols + [col for col in cities_sdf2.columns if '_D20' in col]

# subset columns by time
cities_calyr_19 = cities_sdf2[names_19]
cities_calyr_20 = cities_sdf2[names_20]
cities_calyr_d20 = cities_sdf2[names_D20]


In [19]:
# 10 cities with smallest difference of revenue
cities_calyr_d20[['NAME', 'POPLASTCEN', 'AREA_SQMI', 'TOTAL_D20']].sort_values('TOTAL_D20', ascending=False).head(10)

,NAME,POPLASTCEN,AREA_SQMI,TOTAL_D20
54,St George,76548,77.209900,430010075.0
62,West Jordan,105891,32.306999,299402288.0
20,Layton,69457,22.576200,201867725.0
50,South Salt Lake,24282,6.936550,189977146.0
34,Orem,92993,18.628000,187973628.0
51,Spanish Fork,36232,16.249701,187486973.0
23,Logan,49612,18.485800,186064535.0
63,West Valley City,133796,35.799702,183230415.0
21,Lehi,49315,28.898199,164776229.0
0,American Fork,27413,11.256600,162046226.0


In [20]:
# 10 cities with largest difference of revenue
cities_calyr_d20[['NAME', 'POPLASTCEN', 'AREA_SQMI', 'TOTAL_D20']].sort_values('TOTAL_D20', ascending=True).head(10)

,NAME,POPLASTCEN,AREA_SQMI,TOTAL_D20
45,Salt Lake City,190749,110.734001,-311328076.0
35,Park City,7709,20.341700,-87245688.0
32,North Salt Lake,16930,8.512150,-49944002.0
30,Murray,47311,12.293700,-46516692.0
12,Farmington,18752,10.035100,-31079195.0
43,Roosevelt,6260,6.757070,-27292857.0
27,Moab,5124,4.802460,-13108412.0
52,Springdale,569,4.594880,-8203389.0
59,Vernal,9460,4.632710,-2417569.0
17,Kanab,4413,14.473200,-753309.0


#### Some industries of interest:
- RETAIL-GASOLINE STATIONS (447)
- RETAIL-FOOD & BEVERAGE STORES (445)
- REAL ESTATE, RENTAL & LEASING (53)
- ARTS, ENTERTAINMENT AND RECREATION (71)
- ACCOMMODATION (721)